In [1]:
import numpy as np
from gym import envs
import gym
import random
import sys

In [2]:
# print(envs.registry.all())

In [3]:
#https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py
#https://medium.com/@anirbans17/reinforcement-learning-for-taxi-v2-edd7c5b76869
env = gym.make('Taxi-v2')
env.render()

[2022-05-14 11:55:25,163] Making new env: Taxi-v2


+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



C:\Users\smr99\anaconda3\envs\ps\lib\site-packages\gym\envs\registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


# State Space

|Information| Possible Values | Number of possible Values|
|-----------|-----------------|--------------------------|
| Taxi Row  |0,1,2,3,4        | 5                        |
| Taxi Column  |0,1,2,3,4     | 5                        |
| Destination  |R,G,B,Y (0,1,2,3)      | 4                        |
|Passenger location  |R,G,B,Y, Onboard       | 5         |

# Action Space

- 0: going down (south)
- 1: going up (north)
- 2: going right (east)
- 3: going left (west)
- 4: picking up
- 5: dropping off


In [4]:
action_size = env.action_space.n
print('action_size:{}'.format(action_size))

state_size = env.observation_space.n
print('state_size:{}'.format(state_size))

action_size:6
state_size:500


In [5]:
env.action_space 
num_episodes=50000
test_episodes = 100
steps = 1000

Each state of our agent, which is represented by a vector of values [row of taxi, col of taxi, passenger index, destination index], is consequently encoded with a value between 0 and 499. Namely, we can replicate the location of our previous picture like so:

In [6]:
env = env.unwrapped
state = env.encode(2, 2, 1, 3) 
print("State:", state)
env.s = state
env.render()

State: 247
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [7]:
decode_states=[]
for i in range(500):
    decode_states.append(list(env.decode (i) ))   

As you can see, knowing that our taxi is in position (4,2) and that passenger’s index=3 and drop-off location=2, we can derive that the encoded state is 454. For the following experiment, we will use this starting point, but before diving into that, we need to introduce one last element: the rewarding system.

# Rewarding system
    There is a default per-step reward of -1,
    except for delivering the passenger, which is +20,
    or executing "pickup" and "drop-off" actions illegally, which is -10.

In [8]:
# {action: [(probability, nextstate, reward, done)]
env.P[454]

{0: [(1.0, 454, -1, False)],
 1: [(1.0, 354, -1, False)],
 2: [(1.0, 454, -1, False)],
 3: [(1.0, 434, -1, False)],
 4: [(1.0, 454, -10, False)],
 5: [(1.0, 454, -10, False)]}

# Monte Carlo Agent

In [9]:
from agents import mc_off_policy

In [10]:
from time import process_time 
from IPython.display import clear_output

t1_start = process_time() 
#num_episodes
agent = mc_off_policy.MonteCarloAgent(state_size, action_size,1, env)
agent.initialize_state_actions() 
agent.control()

t1_stop = process_time() 
   
print("Elapsed time:", t1_stop, t1_start) 
print("Elapsed time during the whole program in seconds:", 
                                         t1_stop-t1_start) 

MonteCarloAgent_Q= agent.Q_vals


Episode 0/1Elapsed time: 1.609375 1.5
Elapsed time during the whole program in seconds: 0.109375


In [13]:
agent.save('model_output/final_MC_off_vals_50K.pkl')

agent.load('model_output/final_MC_off_vals_50K.pkl')

In [21]:
 
# env.reset()
# rewards =[]

# for e in range(3):
#     done = False
#     state = env.reset()
#     s = 0
#     r=0
#     while not done:
#         #env.render()
         
#         action = np.argmax(list(MonteCarloAgent_Q[state].values()))
#         new_state, reward, done, info = env.step(action)
#         r+=reward
#         state = new_state

#         if done:
#             rewards.append(r)
#             break
# env.close() 
# MC_rewards=rewards
# print('the score is: {}'.format(sum(rewards)/test_episodes))

# Qlearning

In [9]:
from agents import qlearning

In [11]:
from time import process_time 
from IPython.display import clear_output

# Start the stopwatch / counter  
t1_start = process_time() 
num_episodes=4
agent = qlearning.QLearning(state_size, action_size, env)
agent.initialize_state_actions() 
 
for i_episode in range(num_episodes):
    if i_episode % 100 == 0:
        print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
        sys.stdout.flush()
    agent.control()

    
t1_stop = process_time() 
   
print("Elapsed time:", t1_stop, t1_start) 
print("Elapsed time during the whole program in seconds:", 
                                         t1_stop-t1_start) 

QLearning_Q= agent.Q_vals
#     if i%10 == 9:
#         mcc.evaluate_target_policy()
    
#     if i%100 == 99:
#         mcc.save_your_work()
#         mcc.plot_rewards()

Episode 0/4.***************************
292
***************************
***************************
112
***************************
***************************
92
***************************
***************************
384
***************************
Elapsed time: 1.734375 1.5625
Elapsed time during the whole program in seconds: 0.171875


In [25]:
agent.save('model_output/final_QLearning_50K.pkl')

agent.load('model_output/final_QLearning_50K.pkl')

# SARSA

In [8]:
from agents import sarsa

In [9]:
from time import process_time 
from IPython.display import clear_output
num_episodes=50000
# Start the stopwatch / counter  
t1_start = process_time() 

agent = sarsa.Sarsa(state_size, action_size,num_episodes, env)
agent.initialize_state_actions( )
agent.control() 
 
# for i_episode in range(num_episodes):
#     if i_episode % 100 == 0:
#         print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
#         sys.stdout.flush()
    

    
t1_stop = process_time() 
   
print("Elapsed time:", t1_stop, t1_start) 
print("Elapsed time during the whole program in seconds:", 
                                         t1_stop-t1_start)

Sarsa_Q= agent.Q_vals

Episode 49999/50000. ...0.10001800036000719Elapsed time: 508.34375 1.3125
Elapsed time during the whole program in seconds: 507.03125


In [20]:
agent.save('model_output/final_sarsa_50K.pkl')

agent.load('model_output/final_sarsa_50K.pkl')

# DynaQ

In [11]:
from agents import dynaq

In [9]:
from time import process_time 
from IPython.display import clear_output

# Start the stopwatch / counter  
t1_start = process_time() 

agent = dynaq.DynaQ(state_size, action_size, env)
agent.initialize_state_actions() 

for i_episode in range(num_episodes):
    if i_episode % 100 == 0:
        print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
        sys.stdout.flush()
    agent.control()
    

    

    
t1_stop = process_time() 
   
print("Elapsed time:", t1_stop, t1_start) 
print("Elapsed time during the whole program in seconds:", 
                                         t1_stop-t1_start) 

DynaQ_Q= agent.Q_vals

Episode 0/50000.

KeyboardInterrupt: 

In [33]:
# test_episodes = 100
# env.reset()
# rewards =[]
# r=0
# for e in range(test_episodes):
# #     print('***********************{}********************'.format(e))
#     done = False
#     state = env.reset()
#     r=0
#     while not done:
# #         env.render()
# #         print(state)
# #         print(agent.Q_vals[state])
#         action = np.argmax(DynaQ_Q[state])
#         #print(action)
#         new_state, reward, done, info = env.step(action)
#         r+=reward
#         state = new_state
#         if done:
#             rewards.append(r)
# #             print(r)
#             break
# env.close()    
# DynaQ_rewards=rewards
# print('the score is: {}'.format(sum(rewards)/test_episodes))

In [34]:
agent.save('model_output/final_DynaQ_50K.pkl')

agent.load('model_output/final_DynaQ_50K.pkl')

# dqn

In [12]:
from agents import dqn

ImportError: cannot import name 'Adam' from 'keras.optimizers' (C:\Users\smr99\anaconda3\envs\ps\lib\site-packages\keras\optimizers.py)

In [10]:
batch_size =10
n_episodes = 25000
output_dir = 'model_output/'

In [11]:
agent =  dqn.DQNAgent(4,6, env, decode_states,n_episodes,batch_size, output_dir ,False) # initialise agent

agent.learning( )

agent.save(agent.output_dir + "final_weights.hdf5")

Episode 24999/25000. ...0.10002340004680012

In [12]:
 
agent.load(agent.output_dir + "final_weights.hdf5")

In [13]:
test_episodes = 1
env.reset()
rewards =[]
r=0
for e in range(test_episodes):
#     print('***********************{}********************'.format(e))
    done = False
    state = env.reset()
    r=0
    while not done:
        env.render()
        s =[]
        s.append(  decode_states[state]) 
        act_values = agent.q_estimator.predict(np.array(s))
        action=np.argmax(act_values[0])
#         print(action)
        new_state, reward, done, info = env.step(action)
        r+=reward
        state = new_state
        if done:
            rewards.append(r)
#             print(r)
            break
env.close()    
DQN_rewards=rewards
print('the score is: {}'.format(sum(rewards)/test_episodes))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R:_| : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| :_| : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| :_: : : |
| | : | : |
|Y| : |B: |


In [14]:
# #MC_rewards
# #Sarsa_rewards
# #QLearning_rewards
# #DynaQ_rewards
# import matplotlib.pyplot as plt
# fig= plt.figure(figsize=(10,6))
# ax=fig.add_subplot(111)

# # plt.plot(MC_rewards,c='b', marker="^",ls='--',label='MC',fillstyle='none')
# plt.plot(QLearning_rewards,c='g', ls='--',label='QLearning')
# #plt.plot(Sarsa_rewards,c='k',ls='-',label='Sarsa')
# plt.plot(DynaQ_rewards,c='r', ls='-',label='DynaQ')
# plt.plot(DQN_rewards,c='b', ls='-',label='DQN')

# plt.xlabel('Episodes')
# plt.ylabel('Rewards')
# plt.legend(loc=2)
# plt.show()

In [ ]:
#QLearning_rewards

In [9]:
np.random.rand()

0.5818006863713039

In [15]:
np.random.uniform(0, 1)

0.49492130687032654